In [13]:
%matplotlib inline
import pymongo
from pymongo import MongoClient
import matplotlib
matplotlib.rcParams.update({'font.size': 22})
import matplotlib.pyplot as plt
import numpy
from pprint import pprint
from collections import defaultdict
from datetime import datetime, timedelta
from functools import partial
import time
db = MongoClient('mongodb-condor.icecube.wisc.edu').condor

In [11]:
pipeline = [
    {'$match':{'MATCH_EXP_JOBGLIDEIN_ResourceName':{'$exists': True}}},
    {'$group':{'_id':'$MATCH_EXP_JOBGLIDEIN_ResourceName','count':{'$sum':1}}}
]

In [ ]:
db.command('aggregate', 'things', pipeline=pipeline, explain=True)

In [15]:
start_time = time.time()
pipeline = [
    {'$match':{'MATCH_EXP_JOBGLIDEIN_ResourceName':{'$exists': True}}},
    {'$group':{'_id':'$MATCH_EXP_JOBGLIDEIN_ResourceName','count':{'$sum':1}}}
]
groups = {row['_id']:row['count'] for row in db.condor_history.aggregate(pipeline)}
print(time.time()-start_time)
pprint(len(groups))

OperationFailure: exception: operation was interrupted

In [10]:
db.condor_history.find_one()

{'BytesRecvd': 1889256.0,
 'BytesSent': 179157.0,
 'ClusterId': 7551238,
 'Cmd': '/scratch/simprod/glidein-gpu2/submit/11477/iceprod_9437.1466225051.328931/iceprod.11477.9437.clsim_1.sh',
 'CommittedTime': 442,
 'CompletionDate': 1466226006,
 'CumulativeSlotTime': 442.0,
 'DiskUsage_RAW': 311105,
 'EnteredCurrentStatus': 1466226006,
 'ExecutableSize_RAW': 3,
 'ExitBySignal': False,
 'ExitCode': 0,
 'ExitStatus': 0,
 'GlobalJobId': 'glidein-simprod.icecube.wisc.edu#7551238.0#1466225079',
 'ImageSize_RAW': 107028204,
 'JobCurrentStartDate': 1466225564,
 'JobStartDate': 1466225564,
 'JobStatus': 4,
 'LastJobStatus': 2,
 'LastRemoteHost': 'slot1@3326@kepler19.zeuthen.desy.de',
 'MATCH_EXP_JOBGLIDEIN_ResourceName': 'DESY',
 'NumJobStarts': 1,
 'NumShadowStarts': 1,
 'Owner': 'juancarlos',
 'ProcId': 0,
 'QDate': 1466225079,
 'RemoteSysCpu': 39.0,
 'RemoteUserCpu': 213.0,
 'RemoteWallClockTime': 442.0,
 'RequestCpus': 1,
 'RequestDisk': 2000000,
 'RequestMemory': 2000,
 'Requestgpus': 1,
 'R

In [2]:
def timekey(t):
    return datetime.fromtimestamp(t).strftime('%Y-%m-%d')
def make2list():
    return [0.,0.]                                              

In [19]:
time.mktime((2017,2,14,0,0,0,0,0,0))

1487052000.0

In [16]:
projection = ['MATCH_EXP_JOBGLIDEIN_ResourceName','EnteredCurrentStatus',
              'RemoteWallClockTime','CumulativeSlotTime','NumJobStarts','ExitCode','ExitBySignal','JobStatus']
filter = {'EnteredCurrentStatus':{'$gte':time.mktime((2017,1,7,0,0,0,0,0,0)),
                                  '$lt':time.mktime((2017,2,14,0,0,0,0,0,0))},
         # 'Owner':'i3filter',
          'Requestgpus':1,
         }
# by day
job_restarts = defaultdict(partial(defaultdict,make2list))
cumulative_time = defaultdict(partial(defaultdict,make2list))
success_time = defaultdict(partial(defaultdict,make2list))
removed_time = defaultdict(partial(defaultdict,make2list))
signal_time = defaultdict(partial(defaultdict,make2list))
fail_time = defaultdict(partial(defaultdict,make2list))
start_time = time.time()
for i,row in enumerate(db.condor_history.find(filter=filter,projection=projection)):
    try:
        if 'MATCH_EXP_JOBGLIDEIN_ResourceName' not in row:
            continue
        site = row['MATCH_EXP_JOBGLIDEIN_ResourceName']
        date = timekey(row['EnteredCurrentStatus'])
        if 'NumJobStarts' in row:
            job_restarts[site][date][0] += int(row['NumJobStarts'])
            job_restarts[site][date][1] += 1
        cumulative_time[site][date][0] += float(row['CumulativeSlotTime'])
        cumulative_time[site][date][1] += 1
        if row['JobStatus'] == 4 and (not row['ExitBySignal']) and row['ExitCode'] == 0:
            success_time[site][date][0] += float(row['RemoteWallClockTime'])
            success_time[site][date][1] += 1
        elif row['JobStatus'] == 3:
            removed_time[site][date][0] += float(row['RemoteWallClockTime'])
            removed_time[site][date][1] += 1
        elif row['ExitBySignal']:
            signal_time[site][date][0] += float(row['RemoteWallClockTime'])
            signal_time[site][date][1] += 1
        else:
            fail_time[site][date][0] += float(row['RemoteWallClockTime'])
            fail_time[site][date][1] += 1
    except:
        print(row)
        raise
print(time.time()-start_time)

255.92515349388123


In [6]:
pprint(cumulative_time['WIPAC'].keys())

dict_keys(['2017-01-03', '2017-01-04', '2017-01-05', '2017-01-02', '2017-01-01'])


In [ ]:
db.condor_history.create_index("EnteredCurrentStatus")

In [ ]:
list(db.condor_history.list_indexes())

In [ ]:
db.condor_history.find(filter=filter,projection=projection).explain()